## Importing the database 

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Defining some important variables 

In [2]:
# parameters
learning_rate = 0.0001
epochs = 15
batch_size = 128

# number of samples to calculate validation and accuracy
# decrease this if you're running out of memory
test_valid_size = 256

# network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # dropout (probability to keep units)

weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

## functions to define the model 

In [3]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [4]:
def max_pool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [5]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = max_pool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = max_pool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

## Defining the logits and compiling the model 

In [6]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



## Create a session 

In [7]:
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

        print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))
    test_acc = sess.run(accuracy, feed_dict={
    x: mnist.test.images[:test_valid_size],
    y: mnist.test.labels[:test_valid_size],
    keep_prob: 1.0
    })
    print('Testing accuracy: {}'.format(test_acc))

Epoch  1, Batch 429 -Loss:  1936.7267 Validation Accuracy: 0.722656
Epoch  2, Batch 429 -Loss:  1361.0908 Validation Accuracy: 0.757812
Epoch  3, Batch 429 -Loss:   433.3069 Validation Accuracy: 0.800781
Epoch  4, Batch 429 -Loss:  1101.9415 Validation Accuracy: 0.785156
Epoch  5, Batch 429 -Loss:   246.9187 Validation Accuracy: 0.781250
Epoch  6, Batch 429 -Loss:   379.7260 Validation Accuracy: 0.812500
Epoch  7, Batch 429 -Loss:   217.5623 Validation Accuracy: 0.800781
Epoch  8, Batch 429 -Loss:   235.7073 Validation Accuracy: 0.812500
Epoch  9, Batch 429 -Loss:   293.8632 Validation Accuracy: 0.812500
Epoch 10, Batch 429 -Loss:   332.9141 Validation Accuracy: 0.812500
Epoch 11, Batch 429 -Loss:   190.1723 Validation Accuracy: 0.820312
Epoch 12, Batch 429 -Loss:   391.2674 Validation Accuracy: 0.816406
Epoch 13, Batch 429 -Loss:   181.5151 Validation Accuracy: 0.812500
Epoch 14, Batch 429 -Loss:   349.2806 Validation Accuracy: 0.808594
Epoch 15, Batch 429 -Loss:   215.9090 Validation